**IMPORTANT**  
chatemg源代码输入数据是整型，注意数据类型  
记得在Dataset的实现中加入针对chatemg对数据加偏移量的操作，确保数据中没有负数  
记得转换形状

In [1]:
import yaml

In [2]:
def read_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

In [3]:
config = read_yaml("configs/senic/transformer.yaml")

In [4]:
print(config)

{'dataset': {'name': 'SeNic', 'src': 'datasets/raw/senic_s0', 'sampling_frequency': 200, 'num_channels': 8}, 'data_preprocess': {'enable_filter': False, 'enable_window': True, 'enable_save_windows': False}, 'window': {'type': 'fixed_step', 'window_size': 50, 'step_size': 10, 'windows_dir': 'datasets/windows/senic_s0', 'filename': 'win50_step10.npy'}, 'model': {'name': 'Transformer4EMG', 'N_embed': 256, 'num_attention_heads': 16, 'num_encoder_layers': 4, 'dropout_prob': 0.2, 'enable_norm': True, 'channels_weight_share': False}, 'train': {'val_size': 0.2, 'batch_size': 32, 'enable_shuffle': True, 'criterion': 'Huber', 'num_epochs': 50, 'optimizer': 'AdamW', 'weight_decay': 0.01, 'learning_rate': 0.0001, 'save_checkpoint': False, 'best_model_path': 'checkpoints/senic/win50_stp10/transformer/Nembed128_atthead8_encodelay2.pth', 'enable_lr_decay': True, 'lr_decay_step': 10, 'lr_decay_gamma': 0.5}, 'wandb': {'enable_wandb': True, 'sweep_count': 30, 'project': 'Unlabeled_EMG_Feature_Extraction

In [5]:
from trainer.base_trainer import Trainer

d:\anaconda\envs\potnet\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda\envs\potnet\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
trainer = Trainer(config)

In [7]:
train_loader, val_loader = trainer.load_datasets()

The full windows data is of the shape (361610, 50, 8).


In [8]:
batch = next(iter(train_loader))

In [9]:
inputs, targets = batch

In [10]:
inputs.min()

tensor(-128.)

In [11]:
inputs.max()

tensor(108.)

In [12]:
inputs.shape

torch.Size([32, 49, 8])

In [13]:
targets.shape

torch.Size([32, 8])

In [14]:
targets.dtype

torch.float32

In [15]:
targets = targets.long()

In [16]:
targets.dtype

torch.int64

In [17]:
inputs = inputs.long()

In [18]:
from models.chatemg import ChatEMG

In [19]:
model_config = {
    "dataset": {"num_channels": 8},
    "window": {"window_size": 51},
    "model": {"vocab_size": 256, "n_embed": 256, "n_layer": 12, "bias": False, "dropout": 0.2, "n_head": 8, "token_embedding_type": "basic_sum"}
}

In [20]:
model  = ChatEMG(model_config)

number of parameters: 19.61M


In [21]:
inputs = inputs + 128

In [22]:
inputs.dtype

torch.int64

In [23]:
outputs = model(inputs)

In [24]:
outputs

tensor([[[-1.5892e-01,  7.0319e-02, -2.6995e-01,  ..., -1.0934e-01,
           8.3383e-02, -6.0019e-02],
         [ 7.3120e-03,  9.5174e-02, -5.1272e-02,  ..., -2.2744e-01,
           1.7741e-01,  1.5233e-01],
         [-8.3877e-02,  2.0542e-01, -2.6533e-02,  ...,  1.2038e-01,
           4.4543e-03, -1.1511e-02],
         ...,
         [-7.8506e-02,  2.1880e-01, -1.0634e-01,  ..., -4.4795e-02,
           2.0684e-01,  1.3831e-01],
         [-8.4222e-02,  1.0154e-01, -1.3176e-01,  ..., -3.5816e-02,
           2.0851e-01, -1.2649e-01],
         [ 1.1979e-01, -1.1941e-01,  3.5390e-02,  ..., -9.8970e-02,
           2.4595e-01,  1.8922e-02]],

        [[ 2.9107e-02, -8.2068e-02, -1.3767e-01,  ..., -7.1803e-02,
           3.2093e-01, -3.8027e-02],
         [ 1.3213e-01, -1.8866e-01,  7.5148e-02,  ..., -1.3158e-01,
           3.1331e-02,  9.2463e-02],
         [ 2.1166e-01, -2.6367e-01, -2.2436e-01,  ..., -1.5369e-01,
           9.7096e-02,  1.8860e-01],
         ...,
         [-1.7371e-02, -2

In [26]:
outputs.size()

torch.Size([32, 8, 256])

In [28]:
outputs.dtype

torch.float32

In [29]:
(outputs + 1).dtype

torch.float32

In [20]:
256 // 8

32

In [5]:
config1 = read_yaml("configs/chatemg/chatemg_senic.yaml")

In [6]:
type(config1["dataset"]["offset"])

int

In [ ]:
import torch.nn as nn

In [3]:
import torch

d:\anaconda\envs\potnet\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
a = torch.tensor([1, 2, 3])

In [5]:
a

tensor([1, 2, 3])

In [6]:
a.dtype

torch.int64

In [7]:
print(a.float())

tensor([1., 2., 3.])


In [8]:
a

tensor([1, 2, 3])